# NLP - Mercado Imobiliario

Após instalar as dependencias, executar a instalação do modelo:

```sh
python -m spacy download pt_core_news_md
```

O modelo instalado será o menor, porem existe modelos maiores. Abaixo segue a ordem do menor para o maior:
- pt_core_news_sm
- pt_core_news_md
- pt_core_news_lg


## Setup Inicial

### Termos do mercado imobiliario

In [89]:
termos_imobiliarios = [
    "imovel",
    "quartos",
    "dormitorios",
    "suite",
    "banheiros",
    "vagas",
    "garagem",
    "area",
    "sala",
    "cozinha",
    "varanda",
    "churrasqueira",
    "piscina",
    "academia",
    "condominio",
    "bairro",
]

de_para_imobiliario = {
    "dorm": "dormitorio",
    "dorms": "dormitorios",
}


### Funções de correção e NLP

In [87]:
def pre_correcao(frase: str) -> str:
    def remover_acentos(texto: str) -> str:
        import unicodedata

        nfkd_form = unicodedata.normalize("NFKD", texto)
        only_ascii = nfkd_form.encode("ASCII", "ignore")
        return only_ascii.decode("utf8")

    import re

    frase = remover_acentos(frase)
    frase = re.sub(r"([0-9]+)\s*mil\s", r"\1.000 ", frase)
    frase = re.sub(r"([0-9]+)\s*milhoes\s", r"\1.000.000 ", frase)

    palavras = re.split(r'\s+', frase.lower())
    palavras_imobiliaria = []
    for palavra in palavras:
        achou = False
        palavra_ = palavra.strip()
        for de, para in de_para_imobiliario.items():
            if palavra_ in de.split(","):
                palavras_imobiliaria.append(para)
                achou = True
                break
        if not achou:
            palavras_imobiliaria.append(palavra_)
    return " ".join(palavras_imobiliaria)


def corrigir_ortografia(frase: str) -> str:
    import spellchecker

    corretor = spellchecker.SpellChecker(language="pt")
    palavras = frase.split()
    palavras_corrigidas = []
    for palavra in palavras:
        if correcao := corretor.correction(palavra):
            palavras_corrigidas.append(correcao)
        else:
            palavras_corrigidas.append(palavra)
    frase_corrigida = " ".join(palavras_corrigidas)
    return frase_corrigida


def extrair_tokens(frase: str) -> list[str]:
    import spacy

    nlp = spacy.load("pt_core_news_md")
    doc = nlp(frase)
    tokens = []
    for palavra in doc:
        if (
            palavra.text.lower() in termos_imobiliarios
            or palavra.like_num
            or palavra.pos_ in ["PROPN", "NOUN"]
        ):
            tokens.append(palavra.text)
    return tokens


def extrair_chunks(frase: str) -> list[str]:
    import spacy

    nlp = spacy.load("pt_core_news_md")
    doc = nlp(frase)
    chunks = []
    for chunk in doc.noun_chunks:
        chunks.append(chunk.text)
    return chunks


def retirar_verbos(frase: str) -> tuple[str, list[str]]:
    import spacy

    nlp = spacy.load("pt_core_news_md")
    doc = nlp(frase)
    palavras_sem_verbos = []
    tokens_sem_verbos = []
    for palavra in doc:
        if palavra.pos_ != "VERB":
            palavras_sem_verbos.append(palavra.text)
            tokens_sem_verbos.append(palavra.text)
    frase_sem_verbos = " ".join(palavras_sem_verbos)
    return frase_sem_verbos, tokens_sem_verbos


## Execução

In [88]:
# Testar as funções com uma frase de exemplo
# frase = pre_correcao("queru um imovel 4 dorms")
frase = pre_correcao("quero um imóvel 4 dormitórios e 2 banheiros custando no maximo 2 milhões reais")
frase_corrigida = corrigir_ortografia(frase)
print("Frase corrigida:", frase_corrigida)
tokens = extrair_tokens(frase_corrigida)
print("Tokens:", tokens)
chunks = extrair_chunks(frase_corrigida)
print("Chunks:", chunks)
frase_sem_verbos, tokens_sem_verbos = retirar_verbos(frase_corrigida)
print("Frase sem verbos:", frase_sem_verbos)
print("Tokens sem verbos:", tokens_sem_verbos)

Frase corrigida: quero um imóvel 4 dormitórios e 2 banheiros custando no maximo 2.000.000 reais
Tokens: ['um', 'imóvel', '4', 'dormitórios', '2', 'banheiros', '2.000.000', 'reais']
Chunks: ['um imóvel', '4 dormitórios', '2 banheiros', '2.000.000 reais']
Frase sem verbos: um imóvel 4 dormitórios e 2 banheiros no maximo 2.000.000 reais
Tokens sem verbos: ['um', 'imóvel', '4', 'dormitórios', 'e', '2', 'banheiros', 'no', 'maximo', '2.000.000', 'reais']


## Testes

In [57]:
def teste_01(frase):
    import spacy
    
    nlp = spacy.load("pt_core_news_md")
    doc = nlp(frase)
    for entity in doc.ents:
        print('ent:', entity, entity.label_)
    for chunk in doc.noun_chunks:
        print('chunk:', chunk.text, chunk.start, chunk.end)
    with doc.retokenize() as retokenizer:
        for token in doc:
            if token.like_num:
                next_token = doc[token.i + 1]
                retokenizer.merge(doc[token.i: next_token.i + 1])
    for token in doc:
        print('retokenize:', token.text, token.pos_)

# teste_01('O carro custa 20 mil reais')
teste_01('quero um imóvel 4 dormitórios e 2 banheiros custando no maximo 500 mil reais')

chunk: um imóvel 1 3
chunk: 4 dormitórios 3 5
chunk: 2 banheiros 6 8
chunk: maximo 500 mil reais 10 14


ValueError: [E102] Can't merge non-disjoint spans. 'mil' is already part of tokens to merge. If you want to find the longest non-overlapping spans, you can use the util.filter_spans helper:
https://spacy.io/api/top-level#util.filter_spans